In [ ]:
import os
os._exit(0)



In [4]:
import zipfile
import os
# Unzipping bigData.zip into the data directory
zip_file = "data.zip"  # Replace with the name of your zip file
extracted_folder = "data"  # Target directory to extract files

if os.path.exists(zip_file):
    if not os.path.exists(extracted_folder):
        print(f"Extracting {zip_file} into '{extracted_folder}'...")
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extracted_folder)
        print("Extraction complete.")
    else:
        print(f"'{extracted_folder}' already exists. Skipping extraction.")
else:
    raise FileNotFoundError(f"The ZIP file '{zip_file}' was not found. Please upload it.")



Extracting data.zip into 'data'...
Extraction complete.


In [5]:
### Imports and Globals
import os
import numpy as np
import cv2
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import class_weight
import seaborn as sns
import matplotlib.pyplot as plt

# Constants
SUPPORTED_EXTENSIONS = ['.jpg', '.jpeg', '.png']
IMAGE_SIZE = (128, 128)
FACE_CASCADE = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

### Utility Functions
def clean_images(directory):
    """Cleans and validates image paths, removes unreadable images."""
    image_paths = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(tuple(SUPPORTED_EXTENSIONS)):
                path = os.path.join(root, file)
                img = cv2.imread(path)
                if img is None:
                    print(f"Unreadable image removed: {path}")
                    os.remove(path)
                else:
                    image_paths.append(path)
    print(f"Total valid images found: {len(image_paths)}")
    return image_paths

In [6]:

def detect_and_crop_face(image, size=IMAGE_SIZE):
    """Detects a single face and resizes it."""
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = FACE_CASCADE.detectMultiScale(gray, scaleFactor=1.05, minNeighbors=3, minSize=(20, 20))
    if len(faces) == 1:
        x, y, w, h = faces[0]
        face_crop = image[y:y+h, x:x+w]
        return cv2.resize(face_crop, size)
    return None

def preprocess_images(image_paths, labels):
    """Preprocesses images, applies face detection, and skips bad images."""
    images, valid_labels, valid_paths = [], [], []
    for path, label in zip(image_paths, labels):
        img = cv2.imread(path)
        if img is not None:
            face = detect_and_crop_face(img)
            if face is not None:
                images.append(face / 255.0)
                valid_labels.append(label)
                valid_paths.append(path)  # Keep only valid paths
            else:
                print(f"No face detected: {path}")
        else:
            print(f"Unreadable image skipped: {path}")
    return np.array(images), np.array(valid_labels), valid_paths


In [9]:
# Define the data directory
data_dir = "data/6 Emotions for image classification"

# Clean the dataset and gather valid image paths
print("Cleaning and validating image paths...")
image_paths = clean_images(data_dir)

# Assign placeholder labels for preprocessing (you'll assign real labels later)
placeholder_labels = [os.path.basename(os.path.dirname(path)) for path in image_paths]

# Preprocess the images
print("Preprocessing images and detecting faces...")
images, labels, valid_paths = preprocess_images(image_paths, placeholder_labels)

# Create a DataFrame with valid paths and labels
print("Creating DataFrame with valid paths and labels...")
df = pd.DataFrame({'Path': valid_paths, 'Label': labels})

# Drop unwanted classes
classes_to_drop = ['fear', 'anger', 'pain']  # Classes to exclude
df = df[~df['Label'].isin(classes_to_drop)].reset_index(drop=True)

# Map labels to numeric values
class_mapping = {cls: idx for idx, cls in enumerate(df['Label'].unique())}
df['Label'] = df['Label'].map(class_mapping)

# Confirm the DataFrame
print("Final DataFrame preview:")
print(df.head())


Cleaning and validating image paths...
Total valid images found: 1148
Preprocessing images and detecting faces...
No face detected: data/6 Emotions for image classification/pain/portrait-of-real-chinese-adult-man-with-extreme-expression-and-eyes-closed.jpg
No face detected: data/6 Emotions for image classification/pain/gettyimages-1166803768-612x612.jpg
No face detected: data/6 Emotions for image classification/pain/pexels-photo-1930523 (1).jpeg
No face detected: data/6 Emotions for image classification/pain/pexels-photo-3760043.jpeg
No face detected: data/6 Emotions for image classification/pain/dnz8r16rduh2vxoigq.jpg
No face detected: data/6 Emotions for image classification/pain/dzhob8z0se8dzyrqbe.jpg
No face detected: data/6 Emotions for image classification/pain/dkcdoniucgma3ljau4.jpg
No face detected: data/6 Emotions for image classification/pain/pexels-photo-3807733 (1).jpeg
No face detected: data/6 Emotions for image classification/pain/gettyimages-1385393478-612x612.jpg
No fac

In [10]:
# Map labels to numeric values
class_mapping = {cls: idx for idx, cls in enumerate(df['Label'].unique())}
df['Label'] = df['Label'].map(class_mapping)

In [ ]:





# Preprocess the images and ensure validity
print("Preprocessing images...")
images, labels, valid_paths = preprocess_images(df['Path'], df['Label'])
df = pd.DataFrame({'Path': valid_paths, 'Label': labels})

# Split data into train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.3, stratify=df['Label'], random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, stratify=temp_df['Label'], random_state=42)

# Preprocess train, val, and test sets
X_train, y_train, _ = preprocess_images(train_df['Path'], train_df['Label'])
X_val, y_val, _ = preprocess_images(val_df['Path'], val_df['Label'])
X_test, y_test, _ = preprocess_images(test_df['Path'], test_df['Label'])

# Convert labels to one-hot encoding
y_train_onehot = to_categorical(y_train, num_classes=len(class_mapping))
y_val_onehot = to_categorical(y_val, num_classes=len(class_mapping))
y_test_onehot = to_categorical(y_test, num_classes=len(class_mapping))







Preprocessing images...


In [ ]:
!pip install tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.2 MB/s eta 0:00:00


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Plot Class Distribution
def plot_class_distribution(df, title="Class Distribution"):
    plt.figure(figsize=(8, 5))
    sns.countplot(x='Label', data=df)
    plt.title(title)
    plt.xlabel("Classes")
    plt.ylabel("Count")
    plt.show()

# Visualize Class Distribution
print("Class Distribution in Training Set:")
plot_class_distribution(train_df, "Training Set Class Distribution")

print("Class Distribution in Validation Set:")
plot_class_distribution(val_df, "Validation Set Class Distribution")

print("Class Distribution in Test Set:")
plot_class_distribution(test_df, "Test Set Class Distribution")

# Display Sample Preprocessed Images
def display_sample_images(images, labels, class_mapping):
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle("Sample Preprocessed Images with Labels", fontsize=14)
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i])
        ax.set_title(f"Class: {list(class_mapping.keys())[labels[i]]}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

print("Displaying Sample Preprocessed Images:")
display_sample_images(X_train, y_train, class_mapping)


In [ ]:
### Imports
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Build the base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))
x = base_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.3)(x)  # Added additional Dropout for regularization
output = Dense(len(class_mapping), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

# Step 1: Train with Frozen Base Model
print("Step 1: Training with Frozen Base Model...")
for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Add callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)

# Train the model with frozen base layers
history = model.fit(
    train_generator,
    validation_data=(X_val, y_val_onehot),
    epochs=10,
    callbacks=[early_stopping, lr_scheduler]
)

# Visualize Training Performance for Step 1
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Val Accuracy')
plt.title("Frozen Base Model - Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.title("Frozen Base Model - Loss")
plt.legend()
plt.show()

# Step 2: Gradually Unfreeze and Fine-Tune
print("Step 2: Gradually Unfreezing the Last 20 Layers and Fine-Tuning...")
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Use a smaller learning rate for fine-tuning
optimizer = Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

# Fine-tune the model
history_finetune = model.fit(
    train_generator,
    validation_data=(X_val, y_val_onehot),
    epochs=10,
    callbacks=[early_stopping, lr_scheduler]
)

# Visualize Training Performance for Fine-Tuning
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.plot(history_finetune.history['accuracy'], label='Train Accuracy')
plt.plot(history_finetune.history['val_accuracy'], label='Val Accuracy')
plt.title("Fine-Tuning - Accuracy")
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history_finetune.history['loss'], label='Train Loss')
plt.plot(history_finetune.history['val_loss'], label='Val Loss')
plt.title("Fine-Tuning - Loss")
plt.legend()
plt.show()

# Final Evaluation on Test Set
print("Evaluating on the Test Set...")
test_loss, test_acc = model.evaluate(X_test, y_test_onehot)
print(f"Test Accuracy: {test_acc:.2f}, Test Loss: {test_loss:.2f}")

# Classification Report and Confusion Matrix
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_test_onehot, axis=1)

print("\nClassification Report:")
print(classification_report(y_true_classes, y_pred_classes, target_names=[str(k) for k in class_mapping.keys()]))

cm = confusion_matrix(y_true_classes, y_pred_classes)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, cmap='Blues', fmt='g', xticklabels=class_mapping.keys(), yticklabels=class_mapping.keys())
plt.xlabel('Predicted')
plt.ylabel('True')
plt.title('Confusion Matrix')
plt.show()


In [ ]:
import cv2
import numpy as np

# Load and preprocess the image
image_path = "/content/data/6 Emotions for image classification/happy/smiles_26.jpg"
img = cv2.imread(image_path)

if img is not None:
    # Resize and normalize the image
    img_resized = cv2.resize(img, (128, 128))
    img_normalized = img_resized / 255.0
    img_array = np.expand_dims(img_normalized, axis=0)  # Add batch dimension

    # Predict using the model
    predictions = model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)
    class_name = [name for name, idx in class_mapping.items() if idx == predicted_class[0]][0]

    # Display results
    print(f"Predicted Class: {class_name}")
    print(f"Confidence Scores: {predictions[0]}")
else:
    print(f"Image could not be loaded: {image_path}")


In [ ]:
print("Class Names and Indices:")
for class_name, index in class_mapping.items(): # Use class_mapping instead of label_mapping
    print(f"Class: {class_name}, Index: {index}")

In [ ]:
from sklearn.metrics import f1_score, roc_auc_score

# Weighted F1-Score
weighted_f1 = f1_score(y_true_classes, y_pred_classes, average='weighted')
print(f"Weighted F1-Score: {weighted_f1:.2f}")

# ROC-AUC for Multi-Class
try:
    roc_auc = roc_auc_score(y_test_onehot, y_pred, average="weighted", multi_class="ovr")
    print(f"ROC-AUC Score (Weighted): {roc_auc:.2f}")
except ValueError:
    print("ROC-AUC calculation failed. Ensure proper probabilities and one-hot encoding.")


In [ ]:
def display_predictions_vs_labels(images, y_true, y_pred, class_mapping):
    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle("Predictions vs True Labels", fontsize=14)
    for i, ax in enumerate(axes.flat):
        ax.imshow(images[i])
        true_label = list(class_mapping.keys())[y_true[i]]
        pred_label = list(class_mapping.keys())[y_pred[i]]
        ax.set_title(f"True: {true_label}\nPred: {pred_label}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

print("Displaying Predictions vs True Labels:")
display_predictions_vs_labels(X_test[:10], y_true_classes[:10], y_pred_classes[:10], class_mapping)


In [ ]:
def display_misclassified_images(images, y_true, y_pred, class_mapping):
    misclassified_indices = np.where(y_true != y_pred)[0]
    print(f"Number of Misclassified Images: {len(misclassified_indices)}")

    fig, axes = plt.subplots(2, 5, figsize=(15, 6))
    fig.suptitle("Misclassified Images", fontsize=14)
    for i, ax in enumerate(axes.flat):
        if i >= len(misclassified_indices): break
        idx = misclassified_indices[i]
        ax.imshow(images[idx])
        true_label = list(class_mapping.keys())[y_true[idx]]
        pred_label = list(class_mapping.keys())[y_pred[idx]]
        ax.set_title(f"True: {true_label}\nPred: {pred_label}")
        ax.axis("off")
    plt.tight_layout()
    plt.show()

print("Displaying Misclassified Images:")
display_misclassified_images(X_test, y_true_classes, y_pred_classes, class_mapping)


In [ ]:
print("\n--- Model Training and Fine-Tuning Complete ---")
print(f"Test Accuracy: {test_acc:.2f}")
print(f"Test Loss: {test_loss:.2f}")
print(f"Weighted F1-Score: {weighted_f1:.2f}")
if 'roc_auc' in locals():
    print(f"ROC-AUC Score: {roc_auc:.2f}")
else:
    print("ROC-AUC Score: Not Calculated")
